# Getting started with operators

Qiskit provides sophisticated functionality with operators: [The "Operator Flow" layer in Aqua](https://qiskit.org/documentation/apidoc/qiskit.aqua.operators.html). In this tutorial notebook we will get familiar with the main building blocks of the operator flow layer:

1. State functions
2. Operators and evolutions
3. Measurements and expectations

This tutorial assumes basic familiarty with qubits, quantum superposition and quantum circuits.

There are a few important underlying mathematical concepts:

* An n-qubit State function is a complex function over n binary variables, which we will often refer to as n-qubit binary strings. For example, the traditional quantum “zero state” is a 1-qubit state function, with a definition of f(0) = 1 and f(1) = 0.
* An n-qubit Operator is a linear function taking n-qubit state functions to n-qubit state functions. For example, the Pauli X Operator is defined by f(Zero) = One and f(One) = Zero. Equivalently, an Operator can be defined as a complex function over two n-qubit binary strings, and it is sometimes convenient to picture things this way. By this definition, our Pauli X can be defined by its typical matrix elements, f(0, 0) = 0, f(1, 0) = 1, f(0, 1) = 1, f(1, 1) = 0.
* An n-qubit Measurement is a functional taking n-qubit State functions to complex values. For example, a Pauli Z Measurement can be defined by f(Zero) = 0 and f(One) = 1.

The definitions of these concepts mostly follows the ones in the first two chapters of the book ["The Theory of Quantum Information" by John Watrous](https://cs.uwaterloo.ca/~watrous/TQI/), but don't worry if you don't get them at the first glance: This tutorial is all about getting intuition behind them and the structure of the operator flow layer.

First, let us import the necessary components:

In [1]:
from qiskit.aqua.operators import *
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
import numpy as np

## State functions

Let us start with a single qubit. As we know, a pure state of a qubit is, in general, described as a superposition of basis states $\vert \psi \rangle = \alpha \vert 0\rangle + \beta \vert 1 \rangle$ with complex coefficients $\alpha$ and $\beta$. We can think of this superposition as some mathematical object which puts into correspondence the ket $\vert 0\rangle$ with $\alpha$ and $\vert 1\rangle$ with $\beta$. This is exactly what the state functions in Aqua are. For example the well known minus state $\vert \psi_{-}\rangle=\frac{1}{\sqrt{2}}\vert 0 \rangle - \frac{1}{\sqrt{2}}\vert 1\rangle$ can be thought of as a function mapping the basis ket $\vert 0\rangle$ to $\frac{1}{\sqrt{2}}$ and the basis ket $\vert 1\rangle$ to $-\frac{1}{\sqrt{2}}$. Basis states themselves are particular cases of state sunctions, e.g. the basis state $\vert 0\rangle$ can be thought of as a function mapping the ket $\vert 0\rangle$ to the number $1$ and the ket $\vert 1\rangle$ to the number $0$.

In Aqua, the class `StateFn` can be used to create state functions. Let us create the 1-qubit state function corresponding to the state $\vert \psi_{-}\rangle$

In [2]:
amplitudes = [1, -1]
normalization_coefficient = 1 / np.sqrt(2)
psi_minus = StateFn(amplitudes, coeff=normalization_coefficient)

To access the mapping represented by this state function we can use the function `.eval()`

In [3]:
print(f"The ket |0> is mapped to {psi_minus.eval('0')}")
print(f"The ket |1> is mapped to {psi_minus.eval('1')}")

The ket |0> is mapped to (0.7071067811865475+0j)
The ket |1> is mapped to (-0.7071067811865475+0j)


As you can see we have explicitly specified the normalization coefficient to keep the state normalized. Essentially the Aqua state functions can be thought of and treated as ket vectors, but keep in mind that they are a bit more general mathematical objects and do not impose normalization constraint. At the end of the day, when you create quantum circuits out of state functions and sample them the normalization will be taken care of internally, so you actually don't need to explicitly specify the coefficient every time you create a state function (the default value is `coeff=1`).

If we look at the type of the object `psi_minus`

In [4]:
type(psi_minus)

qiskit.aqua.operators.state_fns.vector_state_fn.VectorStateFn

we can see that it is `VectorStateFn`. This is not much of a surprise, since we built the state function based on a vector of amplitudes. In how many ways can we create a state function? One way is obviously with a vector of complex amplitudes as done above. Another way is by specifying the mapping explicitly with a python dictionary

In [14]:
mapping = {'0': 1, '1': -1}
psi_minus_dict = StateFn(mapping, coeff=normalization_coefficient)
type(psi_minus_dict)

qiskit.aqua.operators.state_fns.dict_state_fn.DictStateFn

`DictStateFn` objects provide an additional convenience where you can omit the zero amplitudes, e.g. the state $\vert 1\rangle$ can be initialized with `{'1': 1}` instead of `{'0': 0, '1': 1}`.

Yet another way to build a state function object is using a quantum circuit

In [19]:
qc = QuantumCircuit(1)
qc.h(0)
qc.z(0)
psi_minus_qc = StateFn(qc)
type(psi_minus_qc)

qiskit.aqua.operators.state_fns.circuit_state_fn.CircuitStateFn

An object used to build a state function is called its _primitive_. The class `StateFn` is a convenient way to create state function objects with various primitives, however the underlying classes `VectorStateFn`, `DictStateFn`, `CircuitStateFn` etc. can be used directly as well, e.g. `DictStateFn({'0': 1, '1': -1})`. The full list of allowed primitives can be studied in the documentation. One important type of primitve is an Aqua operator, which we will cover briefly after we introduce operators and measurements.

To make the usage of state functions easier, Aqua provides the following frequently used state functions as pre-built objects

* $\vert 0 \rangle \rightarrow$ `Zero`
* $\vert 1 \rangle \rightarrow$ `One`
* $\frac{1}{\sqrt{2}} (\vert 0 \rangle + \vert 1 \rangle) \rightarrow$ `Plus`
* $\frac{1}{\sqrt{2}} (\vert 0 \rangle - \vert 1 \rangle) \rightarrow$ `Minus`

E.g. if you need to use the minus state in your code, you can use `Minus`:

In [20]:
print(f"'Minus' maps |0> to {Minus.eval('0')}")
print(f"'Minus' maps |1> to {Minus.eval('1')}")

'Minus' maps |0> to (0.7071067811865476-8.7e-17j)
'Minus' maps |1> to (-0.7071067811865476+8.7e-17j)


Multi-qubit state function objects can be created in a manner identical to one-qubit state functions. As an example let us create a state function object for the 2-qubit Bell state $\vert \Phi^{+} \rangle = \frac{1}{\sqrt{2}}(\vert 00 \rangle + \vert 11 \rangle)$

In [24]:
primitive = {'00': 1, '11': 1}
coeff = 1 / np.sqrt(2)
bell = StateFn(primitive, coeff)

for basis in ['00', '01', '10', '11']:
    print(f"'bell' maps |{basis}> to {bell.eval(basis)}")

'bell' maps |00> to 0.7071067811865475
'bell' maps |01> to 0.0
'bell' maps |10> to 0.0
'bell' maps |11> to 0.7071067811865475


As you know there are a lot of quantum states that can be described as tensor products of one-qubit states. The Bell state we have created above is not a such state, but e.g. the 2-qubit uniform superposition state $\frac{1}{2}(\vert 00 \rangle + \vert 01 \rangle + \vert 10 \rangle + \vert 11 \rangle) = \frac{1}{\sqrt{2}}(\vert 0 \rangle + \vert 1 \rangle) \otimes \frac{1}{\sqrt{2}}(\vert 0 \rangle + \vert 1 \rangle)$ is. Instead of specifying 2-qubit amplitudes to create a state function, as we did for the Bell state, we can use the tensoring capability of state functions to create the uniform superposition state. Tensoring of state functions can be done with the `.tensor()` function (`Plus.tensor(Plus)`) or the `^` symbol (`Plus ^ Plus`). Tensoring can be used also to do tensor powers, i.e. the second operand of tensoring can be an integer instead of a state function  and this will mean the tensor power of the first operand. E.g. let us create a uniform superposition state for five qubits

In [33]:
uniform = Plus ^ 5
uniform.primitive.draw()

┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     └───┘

We have described the introductory basics of Aqua state functions above. Of course they support much more sophisticated functionality which is not covered in this notebook. A few honorable mentions are the functions `sample()` and `to_matrix()` which we encourage the readers to investigate on their own.

## Operators

Now as we can build state functions we can define operations to transform them. Aqua operators do exactly this. Let us define an operator which takes the $\vert - \rangle$ state to the basis state $\vert 1\rangle$. As you know we could accomplish this with a Hadamard gate if we were building a quantum circuit. Let us define an operator based on such a quantum circuit

In [19]:
qc_hadamard = QuantumCircuit(1)
qc_hadamard.h(0)
op_hadamard = CircuitOp(qc_hadamard)

To make the operator `op_hadamard` act on the state `Minus` we need to chain them together. The official term for this in Aqua is _composition_. The symbol `@` can be used to compose the operator with the state function, or the `.compose()` function: they are equivalent

In [26]:
(op_hadamard @ Minus).eval().sample()

{'1': 1.0}

In [27]:
op_hadamard.compose(Minus).eval().sample()

{'1': 1.0}

The state functions and operators have a similar interface. TODO: a few sentences about lazy composition, the type of the object created as a result of composition and the eval function....

TODO: other ways to create operators (one important class - Puli operators)...

TODO: standard operators in operator globals....

## Evolutions

TODO:

TODO: building hamiltonians with operators (talk about SummedOp here as well)....

## Measurements and Expectations

TODO:

## Example

TODO: provide a simple end to end example of initializing some state, evolving it with some hamiltonian and measuring the expectation value of something (e.g. the hamiltonian itself)....